In [1]:
import torch
print(torch.__version__)


2.6.0+cu124


## Required Libraries

Before running this notebook, make sure you have installed the following libraries:


In [2]:
!pip install pytorch-metric-learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [3]:
!pip install torch_geometric
!pip install pyg_lib torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-$(python -c 'import torch; print(torch.__version__)').html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 41.3 MB/s eta 0:00:00


In [4]:
import pickle
import torch
import random
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import math
from math import log

file_path = 'edge_probability_career'


eigen_vector='eigenvector.pkl'

outdegree='outdegree.pkl'

node_to_index_mapping='node_to_index_mapping.pkl'

with open(file_path, 'rb') as file:
    # Load the data from the file
    edge_probability_career = pickle.load(file)

with open(eigen_vector, 'rb') as file:
    # Load the data from the file
    eigenvector = pickle.load(file)


with open(outdegree, 'rb') as file:
    # Load the data from the file
    outdegree = pickle.load(file)


with open(node_to_index_mapping, 'rb') as file:
    # Load the data from the file
    node_to_index_mapping = pickle.load(file)

def outdegree_graph1():
    list_outdegree=[]
    for key in outdegree:
        list_outdegree.append(outdegree[key])
    max_outdegree=max(list_outdegree)
    min_outdegree=min(list_outdegree)

    dict1_out_degree={}
    list_follower=set()
    for key in edge_probability_career:
        dict1={}
        for follower in edge_probability_career[key]:
            list1=[]
            for edge in edge_probability_career[key][follower]:
                source_node_degree=outdegree[key]
                tail_node_degree=outdegree[follower]
                probability=float(source_node_degree+tail_node_degree)/2
                tag=edge[0]
                list1.append((tag,probability))
            dict1[follower]=list1
        dict1_out_degree[key]=dict1
    return dict1_out_degree


# augmenting with eigenvector

def eigenvector1():
    list_eigenvector=[]
    for key in eigenvector:
        list_eigenvector.append(eigenvector[key])

    #indegree_average=sum(list_indegree)/len(list_indegree)
    max_eigenvector=max(list_eigenvector)
    min_eigenvector=min(list_eigenvector)
    #print("")

    dict1_eigenvector={}  # this file will be like edge_probablity career having probaboility as outdegree


    for key in edge_probability_career:
        dict1={}
        for follower in edge_probability_career[key]:
            list1=[]
            for edge in edge_probability_career[key][follower]:
                source_node_degree=eigenvector[key]
                tail_node_degree=eigenvector[follower]
                probability=float(source_node_degree+tail_node_degree)/2
                #probability=float(eigenvector[follower]-min_eigenvector)/(max_eigenvector-min_eigenvector)
                #probability=0.4
                #print("")
                tag=edge[0]
                list1.append((tag,probability))
            dict1[follower]=list1
        dict1_eigenvector[key]=dict1
    return dict1_eigenvector

## Generates augmented graph data


In [7]:
import pickle
import torch
import random
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import math
from math import log
import random

file_path = 'edge_probability_career'


eigen_vector='eigenvector.pkl'

pagerank_file='page_rank.pkl'
with open(pagerank_file, 'rb') as file:
    pagerank = pickle.load(file)

outdegree_file = 'outdegree.pkl'
with open(outdegree_file, 'rb') as file:
    outdegree = pickle.load(file)


indegree_file = 'indegree.pkl'
with open(indegree_file, 'rb') as file:
    indegree = pickle.load(file)

node_to_index_mapping='node_to_index_mapping.pkl'
import numpy as np

node_features="node_features.pkl"


with open('edge_features_v2', 'rb') as file:
    # Load the data from the file
    edge_features = pickle.load(file)

#print(random_array)
with open('edge_probability_career', 'rb') as file:
    # Load the data from the file
    edge_probability_career = pickle.load(file)


with open('node_features.pkl', 'rb') as file:
    # Load the data from the file
    node_features = pickle.load(file)


set_all_nodes = []

# Add keys first
for key in edge_probability_career:
    if key not in set_all_nodes:
        set_all_nodes.append(key)

# Then add followers
for key in edge_probability_career:
    for follower in edge_probability_career[key]:
        if follower not in set_all_nodes:
            set_all_nodes.append(follower)


dict_mapping={} # key node id value index id
index=0

for node in set_all_nodes:
    dict_mapping[node]=index
    index=index+1

list_of_nodes = list(set_all_nodes)

list_node_features=[] # 0 index node feature will be first

for key in dict_mapping:
    try:
        list_node_features.append(node_features[key])
    except:
        # list_node_features.append(np.random.normal(loc=0, scale=1, size=100).astype(np.float32))
        list_node_features.append(np.full((100,), 0.5, dtype=np.float32))

# Load fairness score dictionary
with open('node_fairness_scores.pkl', 'rb') as f:
    fairness_score_dict = pickle.load(f)

# Build fairness_of_nodes list aligned with dict_mapping order
fairness_of_nodes = []
for node in dict_mapping:
    fairness_val = fairness_score_dict[node]  # fallback to random if missing
    fairness_of_nodes.append(fairness_val)


outdegree_graph=outdegree_graph1()


eigenvector_graph=eigenvector1()


# random_augentation_number=2



def func_graph_augmentation(random_augentation_number):
#random_augentation_number=2

    if(random_augentation_number==1):
        dict_outdegree_edge_removed={}
        for key in outdegree_graph:
            dict1={}
            for follower in outdegree_graph[key]:
                list1=[]
                for edge in outdegree_graph[key][follower]:
                    if(edge[1]*0.4>random.uniform(0, 1)):  # 0.5
                    # if(edge[1]*0.4>.2):
                        tag=edge[0]
                        probability=edge[1]
                        list1.append((tag,probability))
                if(len(list1)>0):
                    dict1[follower]=list1
            if(follower in dict1):
                dict_outdegree_edge_removed[key]=dict1
        data_graph=graph_return(dict_outdegree_edge_removed)

        return  data_graph


    if(random_augentation_number==2):

        dict_inf_probablity_removed={}
        for key in edge_probability_career:
            dict1={}
            for follower in edge_probability_career[key]:
                list1=[]
                for edge in edge_probability_career[key][follower]:
                    if(edge[1]*0.8>random.uniform(0, 1)):   # 0.4
                    # if(edge[1]*0.8>.2):
                        tag=edge[0]
                        probability=edge[1]
                        list1.append((tag,probability))
                if(len(list1)>0):
                    dict1[follower]=list1
            if(follower in dict1):
                dict_inf_probablity_removed[key]=dict1
        data_graph=graph_return(dict_inf_probablity_removed)

        return  data_graph
# create the graph data here######################################################################################

def graph_return(graph):
    list_source=[]
    list_target=[]
    edge_features_view=[]
    for key in graph:
        for follower in graph[key]:
            list_source.append(dict_mapping[key])
            list_target.append(dict_mapping[follower])
            edge_features_view.append(edge_features[key][follower])
    edge_index = torch.tensor([list_source, list_target], dtype=torch.long)
    node_features = torch.tensor(np.array(list_node_features), dtype=torch.float)
    result_tensor = torch.stack(edge_features_view)
    result_tensor = torch.squeeze(result_tensor, dim=1)          # this result tensor is the edge features
    fairness_tensor = torch.tensor(fairness_of_nodes, dtype=torch.float64)  # Shape: [num_nodes]
    data_graph = Data(
        x=node_features,
        edge_index=edge_index,
        edge_attr=result_tensor,
        fairness=fairness_tensor,
        node_id=torch.tensor(list(dict_mapping.keys()), dtype=torch.long)
    )
    return data_graph



## **PNAConv-based GNN model** for 500 epochs and stores the learned node embeddings in `embeddings_fairness_mul.pkl`.



In [8]:

import pickle
import torch
import random
from torch_geometric.data import Data
import torch.optim as optim
#from torch_geometric.nn import GCNConv
from torch_geometric.loader import ClusterData, ClusterLoader
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import PNAConv
#import input_graph_creation
import torch.nn.functional as F
import numpy as np
#import input_small_graph_creation
from pytorch_metric_learning.losses import NTXentLoss
import torch_geometric
from torch_geometric.loader import DataLoader
import gc
import psutil
from torch_geometric.nn import GCNConv
from torch_geometric.utils import k_hop_subgraph
# Print memory usage
print(psutil.virtual_memory())
print(torch.__version__)
print(torch_geometric.__version__)
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import PNAConv



class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels,perceptron_hidden_dim,degree,edge_dim):
        super().__init__()
        self.conv1 = PNAConv(in_channels, hidden_channels,["sum","mean","min","max","var","std"],["identity","amplification","attenuation","linear","inverse_linear"],degree,edge_dim)
        self.conv2 = PNAConv(hidden_channels, out_channels,["sum","mean","min","max","var","std"],["identity","amplification","attenuation","linear","inverse_linear"],degree,edge_dim)
        #self.conv1 = GCNConv(in_channels, hidden_channels)
        #self.conv2 = GCNConv(hidden_channels, out_channels)
        self.fc1 = torch.nn.Linear(out_channels, perceptron_hidden_dim)
        self.fc2 = torch.nn.Linear(perceptron_hidden_dim, out_channels)

    def forward(self,data1st_graph,data2nd_graph):

        x, edge_index,edge_attr = data1st_graph.x, data1st_graph.edge_index,data1st_graph.edge_attr

        x = self.conv1(x, edge_index,edge_attr)
        x = torch.relu(x)
        x = self.conv2(x, edge_index,edge_attr)

        x = F.relu(self.fc1(x))
        x_graph1 = self.fc2(x)

        x, edge_index,edge_attr = data2nd_graph.x, data2nd_graph.edge_index,data2nd_graph.edge_attr
        x = self.conv1(x, edge_index,edge_attr)
        x = torch.relu(x)
        x = self.conv2(x, edge_index,edge_attr)

        x = F.relu(self.fc1(x))
        x_graph2 = self.fc2(x)
        return x_graph1,x_graph2
# loss_func = NTXentLoss(temperature=0.10)


def fairness_similarity_loss(embeddings, fairness_scores, multi_hop_followers, node_id_tensor):
    loss = 0.0
    count = 0
    node_id_to_index = {int(nid): idx for idx, nid in enumerate(node_id_tensor.tolist())}

    for infl_id, followers in multi_hop_followers.items():
        if infl_id not in fairness_scores or infl_id not in node_id_to_index:
            continue

        infl_index = node_id_to_index[infl_id]
        infl_embed = embeddings[infl_index]
        target = fairness_scores[infl_id]

        for f_id in followers:
            if f_id not in node_id_to_index:
                continue
            f_index = node_id_to_index[f_id]
            sim = F.cosine_similarity(infl_embed.unsqueeze(0), embeddings[f_index].unsqueeze(0))
            loss += (sim - target).pow(2)
            count += 1

    return loss / (count + 1e-8)


def nt_xent_loss(z1, z2, fairness_weights, temperature=0.1):
    N = min(z1.size(0), z2.size(0))
    z1, z2 = z1[:N], z2[:N]
    f = fairness_weights[:N]

    z = torch.cat([z1, z2], dim=0)  # [2N, D]
    z = F.normalize(z, dim=1)
    sim_matrix = torch.mm(z, z.t()) / temperature  # [2N, 2N]

    mask = torch.eye(2 * N, dtype=torch.bool, device=z.device)

    loss = 0.0
    for idx in range(N):
        # Positive similarity: between z1[idx] and z2[idx]
        pos_sim = torch.exp(F.cosine_similarity(z1[idx].unsqueeze(0), z2[idx].unsqueeze(0)) / temperature)

        # All similarities for z1[idx] and z2[idx] to others (excluding self)
        all_sims_1 = torch.exp(sim_matrix[idx, :])
        all_sims_2 = torch.exp(sim_matrix[idx + N, :])
        all_sims = all_sims_1 + all_sims_2
        all_sims = all_sims[~mask[idx]]  # Remove self-similarity
        denom = all_sims.sum() + 1e-8

        node_loss = -torch.log(pos_sim / denom)
        weight = 1
        loss += (weight) * node_loss

    return loss / N

def train(train_loader, train_loader2, multi_hop_followers, fairness_scores, λ=1.0):
    model.train()
    total_loss_list = []

    for data1st_graph, data2nd_graph in zip(train_loader, train_loader2):
        optimizer.zero_grad()

        h_1, h_2 = model(data1st_graph, data2nd_graph)

        # Match sizes if needed
        if h_2.shape[0] > h_1.shape[0]:
            h_2 = h_2[:h_1.shape[0]]
        else:
            h_1 = h_1[:h_2.shape[0]]

        fairness_weights = data1st_graph.fairness[:h_1.shape[0]].to(h_1.device)

        # Compute NT-Xent Loss
        loss_ntx = nt_xent_loss(h_1, h_2, fairness_weights, temperature=0.1)

        # Compute Fairness Similarity Loss
        concat_embeddings = torch.cat([h_1, h_2], dim=0)
        loss_fair_sim = fairness_similarity_loss(
            concat_embeddings,
            fairness_scores,
            multi_hop_followers,
            data1st_graph.node_id[:concat_embeddings.size(0)]
        )

        total_loss = loss_ntx + λ * loss_fair_sim

        # Step 1: backprop on tensor
        total_loss.backward()
        optimizer.step()

        # Step 2: log float version of loss
        total_loss_list.append(total_loss.item())

    return sum(total_loss_list) / len(total_loss_list)

def save_embeddings(model, loader1, loader2, output_path="embeddings_fairness_mul.pkl"):
    model.eval()
    all_embeddings = []
    all_node_ids = []

    with torch.no_grad():
        for data1, data2 in zip(loader1, loader2):
            # Move to device
            data1 = data1.to(next(model.parameters()).device)
            data2 = data2.to(next(model.parameters()).device)

            h1, h2 = model(data1, data2)

            # Truncate both embeddings and node_id to the same minimum length
            min_len = min(h1.size(0), h2.size(0), data1.node_id.size(0))
            h1 = h1[:min_len]
            h2 = h2[:min_len]
            node_ids = data1.node_id[:min_len]

            # Concatenate embeddings from both graphs
            combined_embedding = torch.cat([h1, h2], dim=1)

            all_embeddings.append(combined_embedding.cpu())
            all_node_ids.extend(node_ids.cpu().numpy())

    final_embeddings = torch.cat(all_embeddings, dim=0).numpy()

    embeddings_dict = {
        "node_ids": all_node_ids,
        "embeddings": final_embeddings,
    }

    with open(output_path, "wb") as f:
        pickle.dump(embeddings_dict, f)

    print(f"Saved {len(all_node_ids)} node embeddings (combined) to '{output_path}'")


def manual_cluster_split(input_graph1, input_graph2, num_parts=10):
    assert input_graph1.num_nodes == input_graph2.num_nodes, "Node count mismatch!"
    num_nodes = input_graph1.num_nodes
    # node_indices = torch.randperm(num_nodes)
    node_indices = torch.arange(num_nodes)
    clusters_1 = []
    clusters_2 = []

    part_size = (num_nodes + num_parts - 1) // num_parts  # ceiling division
    for i in range(0, num_nodes, part_size):
      idx = node_indices[i:i+part_size]

      sub1 = input_graph1.subgraph(idx)
      sub2 = input_graph2.subgraph(idx)

      sub1.node_id = input_graph1.node_id[idx]
      sub2.node_id = input_graph2.node_id[idx]

      sub1.fairness = input_graph1.fairness[idx]
      sub2.fairness = input_graph2.fairness[idx]

      clusters_1.append(sub1)
      clusters_2.append(sub2)

    return clusters_1, clusters_2

def loader_func(input_graph1, input_graph2):
    # Get matching clusters from both graphs
    cluster_subgraphs_1, cluster_subgraphs_2 = manual_cluster_split(input_graph1, input_graph2, num_parts=10)

    # Create data loaders
    train_loader_1 = DataLoader(cluster_subgraphs_1, batch_size=1, shuffle=False)
    train_loader_2 = DataLoader(cluster_subgraphs_2, batch_size=1, shuffle=False)

    return train_loader_1, train_loader_2




in_channels=100
hidden_channels = 80  # Example value, you can change it
#out_channels = 120  # Example value, you can change it
out_channels = 80
#perceptron_hidden_dim = 250
perceptron_hidden_dim = 80
degree=torch.tensor([240, 328,  79,  39,  23,  12,  11,   7,   6,   5,   7,   3,   1,   0,
          2,   0,   0,   0,   1])

edge_dim=100
model = Net(in_channels, hidden_channels, out_channels,perceptron_hidden_dim,degree,edge_dim)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

with open("multi_hop_followers.pkl", "rb") as f:
    multi_hop_followers = pickle.load(f)

with open("node_fairness_scores.pkl", "rb") as f:
    fairness_scores = pickle.load(f)

graph_list=[]
times=1
while(times<3):
    # random_choice = times
    random_choice = np.random.choice([1, 2])
    data_graph=func_graph_augmentation(random_choice)
    graph_list.append(data_graph)
    times=times+1
train_loader,train_loader2=loader_func(graph_list[0],graph_list[1])
for epoch in range(1, 501):

    loss = train(train_loader,train_loader2,multi_hop_followers,fairness_scores)
    print(f"epoch {epoch}, loss: {loss}")

save_embeddings(model, train_loader, train_loader2, output_path="embeddings_fairness_mul.pkl")


svmem(total=13608370176, available=11800911872, percent=13.3, used=1471893504, free=621801472, active=756424704, inactive=11728519168, buffers=127971328, cached=11386703872, shared=2285568, slab=361078784)
2.6.0+cu124
2.6.1
epoch 1, loss: 5.882423019409179
epoch 2, loss: 5.5083636283874515
epoch 3, loss: 5.360521841049194
epoch 4, loss: 5.1496809959411625
epoch 5, loss: 4.899170064926148
epoch 6, loss: 4.6504627704620365
epoch 7, loss: 4.6272365093231205
epoch 8, loss: 4.531843566894532
epoch 9, loss: 4.487884187698365
epoch 10, loss: 4.374628376960755
epoch 11, loss: 4.225434589385986
epoch 12, loss: 4.088127851486206
epoch 13, loss: 4.0844806432724
epoch 14, loss: 3.9774004220962524
epoch 15, loss: 3.726543927192688
epoch 16, loss: 3.6470701932907104
epoch 17, loss: 3.5548145055770872
epoch 18, loss: 3.413642644882202
epoch 19, loss: 3.3182846546173095
epoch 20, loss: 3.212191343307495
epoch 21, loss: 3.1815932989120483
epoch 22, loss: 3.097190523147583
epoch 23, loss: 3.001883888244

In [11]:
import pickle

# Load the pickle file
with open("embeddings_fairness_mul.pkl", "rb") as f:
    data = pickle.load(f)

# Print keys
print("Keys in pickle file:", data.keys())
for i in range(2):
    print(f" embeddings: {data['embeddings'][i]}")


Keys in pickle file: dict_keys(['node_ids', 'embeddings'])
 embeddings: [ 6.17513992e-02 -1.95442036e-01 -4.33062240e-02 -4.52255517e-01
 -6.65501505e-02  1.59940310e-02 -1.10790275e-01 -1.46443799e-01
 -1.68256462e-01 -1.99117698e-04  5.69934025e-04  9.11595076e-02
 -1.15890771e-01 -9.44496170e-02 -2.74717808e-01  2.18510211e-01
 -6.13685064e-02 -5.07791489e-02  7.39697739e-02  3.30987573e-03
  1.26159191e-01  9.74479020e-02 -1.23606429e-01 -2.18232274e-01
 -8.06887001e-02 -1.21948987e-01  1.15545109e-01 -1.51796833e-01
  1.62182719e-01 -1.56608984e-01  1.03809595e-01  4.84202281e-02
 -1.66529492e-01 -1.81111738e-01 -5.73868514e-04  1.25083596e-01
  1.17561162e-01  2.10701972e-01 -1.38994232e-01 -6.49110274e-03
 -6.34399951e-02  7.06671085e-03  2.07076624e-01  5.68343364e-02
 -1.61285415e-01 -1.03544429e-01  1.37668416e-01  2.05318704e-02
 -1.81509212e-01 -7.10483044e-02 -5.15347123e-02 -6.06839061e-02
  2.58787349e-03 -1.29528105e-01  1.85995713e-01  1.03251003e-01
  1.44273430e-01 -